In [1]:
from experiments.ModelLoader import load_vae_model

# from google.colab import drive
# drive.mount('/content/drive')
# !git clone https://github.com/akshitv2/VAE-latent-space-experiment.git
# %cd VAE-latent-space-experiment
# !git checkout celebA-ParamTweaking
# !cp /content/drive/MyDrive/Datasets/img_align_celeba.zip /content/
# !unzip /content/img_align_celeba.zip -d /content/dataset > /dev/null
try:
    import google.colab
    running_in_colab = True
except ImportError:
    running_in_colab = False

if running_in_colab:
    dataset_dir = "/content/dataset"
    out_dir: str = "/content/drive/MyDrive/Temp/outputs"
    checkpoint_dir = "/content/drive/MyDrive/Temp/checkpoints"
else:
    dataset_dir = "G:/Temp"
    out_dir: str = "./outputs/"
    checkpoint_dir = "./experiments/checkpoints"

In [2]:
from torch import GradScaler
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
from tqdm import tqdm
from experiments.Checkpointing import save_checkpoint
from models.VAE import VAE
from modules.Losses import VAEVggLoss
from modules.SaveOutputs import save_reconstructions

torch.backends.cudnn.benchmark = True
torch.cuda.empty_cache()

dataset_dir: str = "./data/raw"
out_dir: str = "./outputs/"
batch_size: int = 64
latent_dim: int = 256
checkpoint_dir = "./experiments/checkpoints"
# epochs: int = 10
lr: float = 3e-4
beta: float = 0.5

current_epoch = 1
os.makedirs(out_dir, exist_ok=True)

Loaded datasets, number of samples:  200573
